# Chain of Responsibility Coding Exercise

You are given a game scenario with classes Goblin and GoblinKing. Please implement the following rules:
- A goblin has base 1 attack/1 defense (1/1), a goblin king is 3/3.
- When the Goblin King is in play, every other goblin gets +1 Attack.
- Goblins get +1 to Defense for every other Goblin in play (a GoblinKing is a Goblin!).

Example:
- Suppose you have 3 ordinary goblins in play. Each one is a 1/3 (1/1 + 0/2 defense bonus).
- A goblin king comes into play. Now every goblin is a 2/4 (1/1 + 0/3 defense bonus from each other + 1/0 from goblin king)

The state of all the goblins has to be consistent as goblins are added and removed from the game.

Here is an example of the kind of test that will be run on the system:

```python
class FirstTestSuite(unittest.TestCase):
    def test(self):
        game = Game()
        goblin = Goblin(game)
        game.creatures.append(goblin)
 
        self.assertEqual(1, goblin.attack)
        self.assertEqual(1, goblin.defense)
```

**Note:** creature removal (unsubscription) does not need to be implemented.

In [1]:
import unittest
from abc import ABC
from enum import Enum

In [2]:
class Creature(ABC):
    def __init__(self, game, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.game = game

    @property
    def attack(self): pass

    @property
    def defense(self): pass

    def query(self, source, query): pass
        
    def __str__(self):
        return f'{self.__class__.__name__} ({self.attack}/{self.defense})'

In [3]:
class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2

class Goblin(Creature):

    def __init__(self, game, attack=1, defense=1):
        super().__init__(game, attack, defense)

    @property
    def attack(self):
        q = Query(self.initial_attack, WhatToQuery.ATTACK)
        for c in self.game.creatures:
            c.query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.initial_defense, WhatToQuery.DEFENSE)
        for c in self.game.creatures:
            c.query(self, q)
        return q.value

    def query(self, source, query):
        if self != source and query.what_to_query == WhatToQuery.DEFENSE:
            query.value += 1
        
class GoblinKing(Goblin):

    def __init__(self, game):
        super().__init__(game, 3, 3)

    def query(self, source, query):
        if self != source and query.what_to_query == WhatToQuery.ATTACK:
            query.value += 1
        else:
            super().query(source, query)

In [4]:
class Query:
    def __init__(self, initial_value, what_to_query):
        self.what_to_query = what_to_query
        self.value = initial_value

class Game:
    def __init__(self):
        self.creatures = []

In [5]:
game = Game()
goblin = Goblin(game)
game.creatures.append(goblin)
print(goblin)

Goblin (1/1)


In [6]:
goblin2 = Goblin(game)
game.creatures.append(goblin2)
print(goblin)

Goblin (1/2)


In [7]:
goblin3 = GoblinKing(game)
game.creatures.append(goblin3)
print(goblin)

Goblin (2/3)


In [9]:
class FirstTestSuite(unittest.TestCase):
    def test(self):
        game = Game()
        goblin = Goblin(game)
        game.creatures.append(goblin)

        self.assertEqual(1, goblin.attack)
        self.assertEqual(1, goblin.defense)

        goblin2 = Goblin(game)
        game.creatures.append(goblin2)

        self.assertEqual(1, goblin.attack)
        self.assertEqual(2, goblin.defense)

        goblin3 = GoblinKing(game)
        game.creatures.append(goblin3)

        self.assertEqual(2, goblin.attack)
        self.assertEqual(3, goblin.defense)

unittest.main(argv=['ignored', '-v'], exit=False)

test (__main__.FirstTestSuite) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
